# read

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import numpy as np; np.random.seed(0)
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('./data/train_df.csv', encoding='cp949')
test  = pd.read_csv('./data/test_df.csv', encoding='cp949')
sub   = pd.read_csv('./data/sample_submission.csv', encoding='cp949')

In [2]:
train.shape, test.shape, sub.shape

((32000, 6), (4640, 5), (4640, 2))

In [3]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT
0,0,1129000014045300,5011000220046300,패션의류,상의,3
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9
3,3,1154500002014200,5011000315087400,식품,농산물,10
4,4,1165000021008300,5011000177051200,식품,가공식품,3


In [4]:
test.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM
0,32000,5013000043028400,1165000021097200,식품,농산물
1,32001,5013000044016100,1154500002066400,식품,농산물
2,32002,5013000205030200,4139000102013200,식품,농산물
3,32003,5013000205030200,4221000040093400,식품,농산물
4,32004,5013000268011400,2726000004017100,식품,농산물


# preprocessing

In [5]:
train['ag'] = 0
test['ag']  = 0

train.loc[train['DL_GD_MCLS_NM'] == '농산물', 'ag'] = 1
test.loc[test['DL_GD_MCLS_NM'] == '농산물', 'ag'] = 1

In [6]:
large_label = dict(train.groupby('DL_GD_LCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)
mid_label   = dict(train.groupby('DL_GD_MCLS_NM').mean().sort_values('INVC_CONT')[['INVC_CONT']].reset_index().values)

train['DL_GD_LCLS_NM'] = train['DL_GD_LCLS_NM'].map(large_label)
test['DL_GD_LCLS_NM']  = test['DL_GD_LCLS_NM'].map(large_label)

train['DL_GD_MCLS_NM'] = train['DL_GD_MCLS_NM'].map(mid_label)
test['DL_GD_MCLS_NM']  = test['DL_GD_MCLS_NM'].map(mid_label)

In [7]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,0
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,0
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,0
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,1
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,0


In [8]:
# 격자공간고유번호 슬라이싱
slice_range = range(3, 11)

for i in slice_range:
    train['send_' + str(i)] = train['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['send_' + str(i)]  = test['SEND_SPG_INNB'].astype('str').str[:i].astype('int64')

for i in slice_range:
    train['rec_' + str(i)] = train['REC_SPG_INNB'].astype('str').str[:i].astype('int64')
    test['rec_' + str(i)]  = test['REC_SPG_INNB'].astype('str').str[:i].astype('int64')

In [9]:
train['eq'] = 0
test['eq']  = 0

train.loc[train['SEND_SPG_INNB'] == train['REC_SPG_INNB'], 'eq'] = 1
test.loc[test['SEND_SPG_INNB'] == test['REC_SPG_INNB'], 'eq'] = 1

In [10]:
threshold1 = 20

temp_send1 = train.groupby('SEND_SPG_INNB').min()
temp_rec1 = train.groupby('REC_SPG_INNB').min()

send_idx1 = temp_send1[temp_send1['INVC_CONT'] >= threshold1].index
rec_idx1  = temp_rec1[temp_rec1['INVC_CONT'] >= threshold1].index

train[(train['SEND_SPG_INNB'].isin(send_idx1)) | (train['REC_SPG_INNB'].isin(rec_idx1))]

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,eq
158,158,5011000319046400,2635000026058300,4.658195,4.297401,33,1,501,5011,50110,501100,5011000,50110003,501100031,5011000319,263,2635,26350,263500,2635000,26350000,263500002,2635000026,0
162,162,5011000373058300,4615000790058400,4.658195,5.780338,36,0,501,5011,50110,501100,5011000,50110003,501100037,5011000373,461,4615,46150,461500,4615000,46150007,461500079,4615000790,0
202,202,5011000767075300,2729000007038300,4.658195,8.196172,25,0,501,5011,50110,501100,5011000,50110007,501100076,5011000767,272,2729,27290,272900,2729000,27290000,272900000,2729000007,0
605,605,1111000025032200,5011000373070400,6.678694,10.375479,37,0,111,1111,11110,111100,1111000,11110000,111100002,1111000025,501,5011,50110,501100,5011000,50110003,501100037,5011000373,0
647,647,4159000310093100,5011000494074200,4.658195,6.130548,22,0,415,4159,41590,415900,4159000,41590003,415900031,4159000310,501,5011,50110,501100,5011000,50110004,501100049,5011000494,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31504,31504,1117000009037100,5011000315059300,6.547718,6.547718,24,0,111,1117,11170,111700,1117000,11170000,111700000,1117000009,501,5011,50110,501100,5011000,50110003,501100031,5011000315,0
31675,31675,5011001053094200,4613000323074400,4.658195,4.297401,20,1,501,5011,50110,501100,5011001,50110010,501100105,5011001053,461,4613,46130,461300,4613000,46130003,461300032,4613000323,0
31915,31915,5011000137023400,2638000022073100,4.658195,6.807151,50,0,501,5011,50110,501100,5011000,50110001,501100013,5011000137,263,2638,26380,263800,2638000,26380000,263800002,2638000022,0
31950,31950,5011000319076400,5011000319076400,4.658195,5.647691,25,0,501,5011,50110,501100,5011000,50110003,501100031,5011000319,501,5011,50110,501100,5011000,50110003,501100031,5011000319,1


In [11]:
train['outlier1'] = 0
test['outlier1'] = 0
train.loc[(train['SEND_SPG_INNB'].isin(send_idx1)) | (train['REC_SPG_INNB'].isin(rec_idx)), 'outlier1'] = 1
test.loc[(test['SEND_SPG_INNB'].isin(send_idx1)) | (test['REC_SPG_INNB'].isin(rec_idx)), 'outlier1'] = 1

In [12]:
train.head()

,index,SEND_SPG_INNB,REC_SPG_INNB,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,eq,outlier
0,0,1129000014045300,5011000220046300,6.678694,3.672897,3,0,112,1129,11290,112900,1129000,11290000,112900001,1129000014,501,5011,50110,501100,5011000,50110002,501100022,5011000220,0,0
1,1,1135000009051200,5011000178037300,5.407921,4.444134,3,0,113,1135,11350,113500,1135000,11350000,113500000,1135000009,501,5011,50110,501100,5011000,50110001,501100017,5011000178,0,0
2,2,1135000030093100,5011000265091400,6.678694,10.375479,9,0,113,1135,11350,113500,1135000,11350000,113500003,1135000030,501,5011,50110,501100,5011000,50110002,501100026,5011000265,0,0
3,3,1154500002014200,5011000315087400,4.658195,4.297401,10,1,115,1154,11545,115450,1154500,11545000,115450000,1154500002,501,5011,50110,501100,5011000,50110003,501100031,5011000315,0,0
4,4,1165000021008300,5011000177051200,4.658195,6.807151,3,0,116,1165,11650,116500,1165000,11650000,116500002,1165000021,501,5011,50110,501100,5011000,50110001,501100017,5011000177,0,0


# encoding

In [13]:
train.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)
test.drop(['index', 'SEND_SPG_INNB', 'REC_SPG_INNB'], axis=1, inplace=True)

In [14]:
train.head()

,DL_GD_LCLS_NM,DL_GD_MCLS_NM,INVC_CONT,ag,send_3,send_4,send_5,send_6,send_7,send_8,send_9,send_10,rec_3,rec_4,rec_5,rec_6,rec_7,rec_8,rec_9,rec_10,eq,outlier
0,6.678694,3.672897,3,0,112,1129,11290,112900,1129000,11290000,112900001,1129000014,501,5011,50110,501100,5011000,50110002,501100022,5011000220,0,0
1,5.407921,4.444134,3,0,113,1135,11350,113500,1135000,11350000,113500000,1135000009,501,5011,50110,501100,5011000,50110001,501100017,5011000178,0,0
2,6.678694,10.375479,9,0,113,1135,11350,113500,1135000,11350000,113500003,1135000030,501,5011,50110,501100,5011000,50110002,501100026,5011000265,0,0
3,4.658195,4.297401,10,1,115,1154,11545,115450,1154500,11545000,115450000,1154500002,501,5011,50110,501100,5011000,50110003,501100031,5011000315,0,0
4,4.658195,6.807151,3,0,116,1165,11650,116500,1165000,11650000,116500002,1165000021,501,5011,50110,501100,5011000,50110001,501100017,5011000177,0,0


# modeling

In [15]:
X = train.drop('INVC_CONT', axis=1)
y = train['INVC_CONT']

In [16]:
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

model = CatBoostRegressor(random_state=0, random_strength=0)
model.fit(X, y)
pd.DataFrame({'col':X.columns, 'fi':model.feature_importances_})

Learning rate set to 0.070793
0:	learn: 5.6270548	total: 277ms	remaining: 4m 36s
1:	learn: 5.5140616	total: 340ms	remaining: 2m 49s
2:	learn: 5.3895688	total: 446ms	remaining: 2m 28s
3:	learn: 5.2749795	total: 632ms	remaining: 2m 37s
4:	learn: 5.1712330	total: 702ms	remaining: 2m 19s
5:	learn: 5.0777758	total: 803ms	remaining: 2m 12s
6:	learn: 4.9947807	total: 1.09s	remaining: 2m 35s
7:	learn: 4.9196517	total: 1.2s	remaining: 2m 28s
8:	learn: 4.8530775	total: 1.27s	remaining: 2m 19s
9:	learn: 4.7930768	total: 1.4s	remaining: 2m 18s
10:	learn: 4.7392671	total: 1.52s	remaining: 2m 16s
11:	learn: 4.6916057	total: 1.6s	remaining: 2m 11s
12:	learn: 4.6480907	total: 1.72s	remaining: 2m 10s
13:	learn: 4.6094722	total: 1.83s	remaining: 2m 9s
14:	learn: 4.5744912	total: 1.91s	remaining: 2m 5s
15:	learn: 4.5427730	total: 2.04s	remaining: 2m 5s
16:	learn: 4.5139821	total: 2.11s	remaining: 2m 2s
17:	learn: 4.4872204	total: 2.21s	remaining: 2m
18:	learn: 4.4628650	total: 2.37s	remaining: 2m 2s
19:	

,col,fi
0,DL_GD_LCLS_NM,1.620104
1,DL_GD_MCLS_NM,12.775583
2,ag,0.082166
3,send_3,0.552848
4,send_4,4.261231
5,send_5,5.002457
6,send_6,0.000000
7,send_7,4.002474
8,send_8,6.424931
9,send_9,6.027390


In [ ]:
# drop_cols = ['ag', 'send_3', 'send_6', 'rec_3', 'rec_5', 'rec_6', 'rec_7', 'rec_10', 'eq']
# X.drop(drop_cols, axis=1, inplace=True)
# X.head()

In [ ]:
# model = CatBoostRegressor(random_state=0, random_strength=0)
# model.fit(X, y)
# pd.DataFrame({'col':X.columns, 'fi':model.feature_importances_})

In [17]:
from sklearn.model_selection import GridSearchCV

model = CatBoostRegressor(random_state=0, random_strength=0)

param = {
    
}

gscv = GridSearchCV(
    estimator=model, param_grid=param, scoring='neg_root_mean_squared_error', 
    refit=True, cv=5
)

gscv.fit(X, y)

print(gscv.best_score_)

Learning rate set to 0.068341
0:	learn: 5.3240070	total: 63.9ms	remaining: 1m 3s
1:	learn: 5.2130246	total: 119ms	remaining: 59.5s
2:	learn: 5.1084928	total: 221ms	remaining: 1m 13s
3:	learn: 5.0155771	total: 327ms	remaining: 1m 21s
4:	learn: 4.9282843	total: 386ms	remaining: 1m 16s
5:	learn: 4.8524587	total: 469ms	remaining: 1m 17s
6:	learn: 4.7839349	total: 538ms	remaining: 1m 16s
7:	learn: 4.7193211	total: 641ms	remaining: 1m 19s
8:	learn: 4.6633960	total: 725ms	remaining: 1m 19s
9:	learn: 4.6128019	total: 787ms	remaining: 1m 17s
10:	learn: 4.5669970	total: 885ms	remaining: 1m 19s
11:	learn: 4.5251106	total: 969ms	remaining: 1m 19s
12:	learn: 4.4874449	total: 1.04s	remaining: 1m 19s
13:	learn: 4.4529389	total: 1.11s	remaining: 1m 18s
14:	learn: 4.4218197	total: 1.18s	remaining: 1m 17s
15:	learn: 4.3934835	total: 1.25s	remaining: 1m 16s
16:	learn: 4.3676428	total: 1.31s	remaining: 1m 15s
17:	learn: 4.3440403	total: 1.36s	remaining: 1m 14s
18:	learn: 4.3226549	total: 1.49s	remaining: 

In [18]:
pred = gscv.predict(test)
sub['INVC_CONT'] = pred

In [19]:
sub.head()

,index,INVC_CONT
0,32000,4.396690
1,32001,4.392931
2,32002,4.759586
3,32003,4.559870
4,32004,4.497850


In [20]:
# sub.to_csv('./sub/logistics_09.csv', index=False)